In [5]:
import numpy as np

In [6]:
file_name = "../data/demis_hassabis.vtt"
with open(file_name, "r") as f:
    lines = f.readlines()

In [7]:
lines[0:5]

['WEBVTT\n',
 '\n',
 '00:00.000 --> 00:03.480\n',
 ' The following is a conversation with Demis Hassabis,\n',
 '\n']

In [8]:
text = [item.strip() for idx, item in enumerate(lines) if idx%3 == 0]
text = text[1:]
print(len(text))

3105


In [23]:
timeline = [item.strip() for idx, item in enumerate(lines) if idx%3 == 2]
timeline[0:5]

['00:00.000 --> 00:03.480',
 '00:03.480 --> 00:06.720',
 '00:06.720 --> 00:08.600',
 '00:08.600 --> 00:12.200',
 '00:12.200 --> 00:14.120']

In [10]:
len(timeline)

3105

In [11]:
assert len(text)==len(timeline)

In [12]:
text = " ".join(text)

In [13]:
from summa.summarizer import summarize
summary = summarize(text, ratio=0.01, split=True)
summary

["And I still think something like the Turing test will, at the end of the day, be the thing that truly impresses other humans so that you can have a close friend who's an AI system.",
 "I just, I think there'll be a kind of humor in the AI systems of the future looking back to this conversation and thinking about the Turing test and also thinking about by that time, they would know which year they were finally able to sort of cross the threshold of human level intelligence and think how funny it is that we humans were still confused about this whole problem that has been long ago been solved.",
 'So whether it was graphics with people like John Carmack and Quake and those kinds of things or AI, I think actually all the action was going on in games.',
 "And I think that building AI, one of the reasons I wanna build AI, and I've always wanted to is, I think by building an intelligent artifact like AI, and then comparing it to the human mind, that will help us unlock the uniqueness and t

In [14]:
len(text.split())

23448

In [15]:
len(summary)

12

In [ ]:
'''xclass whisper_unit(object):
    def __init__(self, start, end, text):
        self.start = start
        self.end = end
        self.text = text
        
    def '''

In [16]:
import time

def isTime(input):
    try:
        time.strptime(input, '%H:%M:%S.%f') 
        return True
    except ValueError:
        return False
    
isTime("01:13.120")

def convert_time(ip):
    if ip.count(':') == 1:
        ip = "00:" + ip
    return time.strptime(ip, '%H:%M:%S.%f')

def convert_string(timeval):
    return time.strftime('%H:%M:%S', timeval)

In [17]:
transcripts = []
for idx, line in enumerate(lines):
    if "-->" in line:
        timeval = line.strip()
        textval = lines[idx+1:idx+2][0].strip()
        #print(timeval, textval[0].strip())
        
        vals = timeval.split("-->")
        start = vals[0].strip()
        end = vals[1].strip()
        start_time = convert_time(start)
        end_time = convert_time(end)
        transcript = {
            "timewindow": timeval,
            "text" : textval,
            "start": convert_string(start_time),
            "end": convert_string(end_time)
        }
        transcripts.append(transcript)

In [18]:
time.strptime("00:03.480", '%M:%S.%f') 

time.struct_time(tm_year=1900, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=3, tm_wday=0, tm_yday=1, tm_isdst=-1)

In [20]:
transcripts[0:5]

[{'timewindow': '00:00.000 --> 00:03.480',
  'text': 'The following is a conversation with Demis Hassabis,',
  'start': '00:00:00',
  'end': '00:00:03'},
 {'timewindow': '00:03.480 --> 00:06.720',
  'text': 'CEO and co founder of DeepMind,',
  'start': '00:00:03',
  'end': '00:00:06'},
 {'timewindow': '00:06.720 --> 00:08.600',
  'text': 'a company that has published and built',
  'start': '00:00:06',
  'end': '00:00:08'},
 {'timewindow': '00:08.600 --> 00:12.200',
  'text': 'some of the most incredible artificial intelligence systems',
  'start': '00:00:08',
  'end': '00:00:12'},
 {'timewindow': '00:12.200 --> 00:14.120',
  'text': 'in the history of computing,',
  'start': '00:00:12',
  'end': '00:00:14'}]

In [21]:
summ_transcripts = []
for summ in summary:
    start_index = -1
    end_index = -1
    init_text = " ".join(summ.split()[0:5])
    end_text = " ".join(summ.split())[-5:]
    for index, item in enumerate(transcripts):
        if init_text in item['text']:
            start_index = index
            break
    if start_index > 0:
        for index, item in enumerate(transcripts[start_index:start_index+20]):
            if end_text in item['text']:
                end_index = start_index + index
                break
    
    if start_index > -1  and end_index > -1:
        summ_transcript = {
            "summary_text": summ,
            "start_index": start_index,
            "end_index" : end_index
        }
        summ_transcripts.append(summ_transcript)
    else:
        print("not found")

In [22]:
summ_transcripts

[{'summary_text': "And I still think something like the Turing test will, at the end of the day, be the thing that truly impresses other humans so that you can have a close friend who's an AI system.",
  'start_index': 143,
  'end_index': 146},
 {'summary_text': "I just, I think there'll be a kind of humor in the AI systems of the future looking back to this conversation and thinking about the Turing test and also thinking about by that time, they would know which year they were finally able to sort of cross the threshold of human level intelligence and think how funny it is that we humans were still confused about this whole problem that has been long ago been solved.",
  'start_index': 174,
  'end_index': 182},
 {'summary_text': 'So whether it was graphics with people like John Carmack and Quake and those kinds of things or AI, I think actually all the action was going on in games.',
  'start_index': 339,
  'end_index': 341},
 {'summary_text': "And I think that building AI, one of th

In [1]:
import time
class KeyMoments(object):

    def __init__(self, transcript=None):
        self.wh_transcript = transcript
        
    def _convert_time(self, window_time):
        if window_time.count(':') == 1:
            window_time = "00:" + window_time
        return time.strptime(window_time, '%H:%M:%S.%f')

    def _convert_string(self, timeval):
        return time.strftime('%H:%M:%S', timeval)
    
    def _convert_to_secs(self, timeval):
        t = self._convert_time(timeval)
    
    def _process_transcript(self):
        transcripts = []
        for idx, line in enumerate(self.wh_transcript):
            if "-->" in line:
                time_window = line.strip()
                window_text = self.wh_transcript[idx+1:idx+2][0].strip()

                windows = time_window.split("-->")
                start = windows[0].strip()
                end = windows[1].strip()
                start_time = self._convert_time(start)
                end_time = self._convert_time(end)
                transcript = {
                    "time_window": time_window,
                    "text" : window_text,
                    "start_time": self._convert_string(start_time),
                    "end_time": self._convert_string(end_time)
                }
                transcripts.append(transcript)
        return transcripts
    
    def _summarize(self, text):
        from summa.summarizer import summarize
        summary = summarize(text, ratio=0.01, split=True)
        return summary
    
    def _process_summary(self, summary, transcripts):
        summ_transcripts = []
        for summ in summary:
            start_index = -1
            end_index = -1
            init_text = " ".join(summ.split()[0:5])
            end_text = " ".join(summ.split())[-5:]
            for index, item in enumerate(transcripts):
                if init_text in item['text']:
                    start_index = index
                    break
            if start_index > 0:
                for index, item in enumerate(transcripts[start_index:start_index+20]):
                    if end_text in item['text']:
                        end_index = start_index + index
                        break

            if start_index > -1  and end_index > -1:
                summ_transcript = {
                    "key_moment": summ,
                    "start_index": start_index,
                    "end_index" : end_index,
                    "start_time": transcripts[start_index]['start_time'],
                    "end_time": transcripts[end_index]['end_time'],
                }
                summ_transcripts.append(summ_transcript)
            else:
                print("not found")
        return summ_transcripts
    
    def get_key_moments(self, transcripts):
        if transcripts:
            self.wh_transcript = transcripts
            
        processed = self._process_transcript()
        
        all_text = [item['text'] for item in processed]
        all_text = " ".join(all_text)
        summaries = self._summarize(all_text)
        keymoments = self._process_summary(summaries, processed)
        
        return keymoments
        

In [ ]:
moments = KeyMoments()
moments.get_key_moments(lines)

# Create User Inferface

In [2]:
import gradio as gr

def create_key_moments(url, file):
    key_moments = []
    if file:
        print(file.name)
        with open(file.name, "r") as f:
            text = f.readlines()
            
        print(text[0:5])
        
        moments = KeyMoments()
        key_moments = moments.get_key_moments(text)
        
    return key_moments[0:3]
    

In [3]:
demo = gr.Interface(fn=create_key_moments, description="Key Moments Demo", inputs=[
        gr.Text(label="URL (YouTube, etc.)"),
        gr.File(type="file", label="Upload vtt file"), 
    ], outputs=[
        gr.Text(label="Segments")
    ])

In [4]:
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/var/folders/vk/c62whj215n12n7wh_5z7d4w00000gn/T/demis_hassabisaep6tcwt.vtt
['WEBVTT\n', '\n', '00:00.000 --> 00:03.480\n', ' The following is a conversation with Demis Hassabis,\n', '\n']
/var/folders/vk/c62whj215n12n7wh_5z7d4w00000gn/T/demis_hassabisukc7wc2v.vtt
['WEBVTT\n', '\n', '00:00.000 --> 00:03.480\n', ' The following is a conversation with Demis Hassabis,\n', '\n']
Keyboard interruption in main thread... closing server.


In [ ]:
demo.close()